In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

In [ ]:
df={}
data_master=pd.DataFrame()
for i in range(2,319):
    x='data'+str(i)+'.txt'
    df[i]=pd.read_csv(x,sep=',',header=None)
    df[i].drop(columns=[0,4],inplace=True)
    data_master=data_master.append(df[i])

In [ ]:
def smoth(value):
    window_length = 101

    poly_order = 4

    # Apply the Savitzky-Golay filter
    y1 = savgol_filter(value[1], window_length, poly_order)
    y2 = savgol_filter(value[2], window_length, poly_order)
    y3 = savgol_filter(value[3], window_length, poly_order)
    return y1,y2,y3


In [ ]:
a,b,c={},{},{} #just like 1,2,3 ==> a,b,c are the three phases for smoothen graph.
for i in range(2,319):
    a[i],b[i],c[i]=smoth(df[i])

In [ ]:
def minima_maxima(value):
    peak_indices, _= find_peaks(value)
    minima_indices, _= find_peaks(-value)
    return peak_indices,minima_indices

In [ ]:
maxima={}# from here we have maximas and minimas for all the graphs
minima={}
for i in range(2,319):
    maxima[i],minima[i]=minima_maxima(a[i])

In [ ]:
def outlier(value):
    q1=np.quantile(value,0.25)
    q3=np.quantile(value,0.75)
    iqr=q3-q1
    ul=q3+(1.5*iqr)
    ll=q1-(1.5*iqr)
    return ul,ll

In [ ]:
uplimit={}
lowlimit={}
for i in range(2,319):
    uplimit[i],lowlimit[i]=outlier(a[i])
     #checking outliers for all the files
    if (a[i]>uplimit[i]).any() or (a[i]<lowlimit[i]).any():
        print(i,'.txt file has outliers')

In [ ]:
def phase_defected_peaks(value):
    u_peak,l_peak=outlier(value)
    x=np.zeros(len(value))
    x[np.where(value<l_peak)]=1
    x[np.where(value>u_peak)]=1
    df_peak=pd.DataFrame(data=[value,x]).T
    df_peak.columns=['Phase_Peak','Fault']
    return df_peak


def phase_defected_crests(value):
    
    u_crest,l_crest=outlier(value)
    y=np.zeros(len(value))
    y[np.where(value>u_crest)]=1
    y[np.where(value>u_crest)]=1
    df_crest=pd.DataFrame(data=[value,y]).T
    df_crest.columns=['Phase_Crest','Fault']
    
    return df_crest

In [ ]:
d_peak={}
d_crest={}
d_peak_master=pd.DataFrame()
d_crest_master=pd.DataFrame()
for i in range(2,319):
    d_peak[i]=phase_defected_peaks(a[i][maxima[i]])
    d_crest[i]=phase_defected_crests(a[i][minima[i]])
    d_peak_master=d_peak_master.append(d_peak[i])
    d_crest_master=d_crest_master.append(d_crest[i])